In [6]:
import pandas as pd



In [8]:
respondieron = pd.read_csv('respondieron.csv')


In [10]:
respondieron

,cond. de actividad,PEA
0,3,2
1,1,1
2,3,2
3,1,1
4,3,2
...,...,...
14642,3,2
14643,2,1
14644,1,1
14645,1,1


In [14]:
norespondieron = pd.read_csv('norespondieron.csv')

In [16]:
norespondieron

,cond. de actividad,ano4,sexo,edad,estado civil,seguro médico,nivel educativo,categoria inactivo,ipcf
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
respondieronconaños = pd.read_csv('respondieron_con_años.csv')

In [22]:
respondieronconaños

,cond. de actividad,PEA,ano4
0,3.0,2.0,2004.0
1,1.0,1.0,2004.0
2,3.0,2.0,2004.0
3,1.0,1.0,2004.0
4,3.0,2.0,2004.0
...,...,...,...
14642,3.0,2.0,2024.0
14643,2.0,1.0,2024.0
14644,1.0,1.0,2024.0
14645,1.0,1.0,2024.0


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [24]:
# Suponiendo que los "desocupados" eran codificados con un 2 originalmente antes de reemplazar
# Si ya reemplazaste y no tienes forma de distinguir desocupados dentro de activos, esto es una suposición
respondieron["desocupado"] = (respondieronconaños["PEA"] == 1).astype(int)


In [32]:
print(respondieronconaños["ano4"].unique()) #estan bien cargados los dos años (2004) y (2024)


[2004. 2024.]


In [40]:
for año in respondieronconaños["ano4"].unique():
    print(f"\nProcesando año: {año}")
    
  
    df_anual = respondieronconaños[respondieronconaños["ano4"] == año]
    
    

    X = df_anual.drop(columns=["ano4", "PEA", "cond. de actividad"], errors='ignore')
    
   
    X = X.copy() 
    X["intercept"] = 1
    
  
    X_train, X_test = train_test_split(X, train_size=0.7, random_state=444)
    
    print(f"Año {año}:")
    print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")



Procesando año: 2004.0
Año 2004.0:
Train shape: (5352, 1), Test shape: (2295, 1)

Procesando año: 2024.0
Año 2024.0:
Train shape: (4900, 1), Test shape: (2100, 1)


In [46]:
respondieronconaños["desocupado"] = (respondieronconaños["PEA"] == 1).astype(int)

In [48]:
# Supongamos que 'anio' es el nombre de la columna del año
for año in respondieronconaños["ano4"].unique():
    print(f"\nProcesando año: {año}")
    
    # Filtrar por año
    df_anual = respondieronconaños[respondieronconaños["ano4"] == año]
    
    # Variable dependiente
    y = df_anual["desocupado"]
    
    # Variables independientes (puedes excluir columnas como 'desocupado', 'anio', etc.)
    X = df_anual.drop(columns=["desocupado", "ano4", "PEA", "cond. de actividad"], errors='ignore')
    
    # Agregar columna de unos
    X = X.copy()  # para evitar SettingWithCopyWarning
    X["intercept"] = 1
    
    # Separar en train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=444)
    
    print(f"Año {año}:")
    print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")



Procesando año: 2004.0
Año 2004.0:
Train shape: (5352, 1), Test shape: (2295, 1)

Procesando año: 2024.0
Año 2024.0:
Train shape: (4900, 1), Test shape: (2100, 1)


In [50]:
common_cols = X_train.columns.intersection(X_test.columns)


In [52]:
mean_train = X_train[common_cols].mean()
mean_test = X_test[common_cols].mean()

In [54]:
diff_table = pd.DataFrame({
    "Media Entrenamiento": mean_train,
    "Media Testeo": mean_test,
    "Diferencia Absoluta": (mean_train - mean_test).abs()
})

In [56]:
diff_table = diff_table.sort_values("Diferencia Absoluta", ascending=False)
print(diff_table)

           Media Entrenamiento  Media Testeo  Diferencia Absoluta
intercept                  1.0           1.0                  0.0
